<a href="https://colab.research.google.com/github/julurisaichandu/nlp/blob/main/notebook_05_ngram_markov_assumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook 5: N-gram Language Models
===============

CS 6120 Natural Language Processing, Amir



Saichandu Juluri

Saving notebooks as pdfs
----------

Feel free to add cells to this notebook as you wish. Make sure to leave **code that you've written** and any **answers to questions** that you've written in your notebook. Turn in your notebook as a pdf at the end of lecture's day.


To convert your notebook to a pdf for turn in, you'll do the following:
1. Kernel -> Restart & Run All (clear your kernel's memory and run all cells)
2. File -> Download As -> .html -> open in a browser -> print to pdf

(The download as pdf option doesn't preserve formatting and output as nicely as taking the step "through" html, but will do if the above doesn't work for you.)

Task 1: Implement an N-gram Language Model
-------

Recall that a language model estimates the probability of a sentence as

$P(w_1, \ldots, w_N) = \prod_i^N P(w_i | w_{1},\ldots,w_{i-1})$

N-gram Language Models approximate the conditional probabilities using a Markov Assumption

$P(w_i| w_1, \ldots, w_{i-1}) \approx P(w_i | w_{i-n+1}, \ldots, w_{i-1})$

e.g., for bi-grams we get

$P(w_i| w_1, \ldots, w_{i-1}) \approx P(w_i | w_{i-1})$

Training entails estimating the n-gram probabilities

$P(w_i | w_{i-n+1}, \ldots, w_{i-1}) = \frac{\text{count}(w_{i-n+1}, \ldots, w_{i-1}w_i)}{\text{count}(w_{i-n+1}, \ldots, w_{i-1})}$

for bi-grams we get

$P(w_i|w_{i-1}) =\frac{\text{count}(w_{i-1}w_i)}{\text{count}(w_{i-1})}$

Note that for unigrams we compute the unconditional word probabilities

$P(w_i) =\frac{\text{count}(w_i)}{\sum_{j \in V}\text{count}(w_{j})}$

We may also want to use Laplace Smoothing (like we did for Naive Bayes training)


We measure model performance by calculating perplexity on a test set

$PP(w_1, \ldots, w_N) = P(w_1, \ldots, w_N)^{-1/N}$


<!-- To make this calculation more stable we can operate in log space

$\hat{y} = \arg\max_{y \in \{0, 1\}} \log P(y) + \sum_{i=1}^{n} \log P(x_i|y)$

Training entails estimating:

1. class priors

$P(y) = \frac{N_y}{N}$
where $N_y$ is the number of documents with class $y$ and $N$ is the total number of documents


2. class conditional word probabilities

$P(x_i|y) = \frac{\text{count}(x_i,y)}{\sum_{x \in V} \text{count}(x,y)}$


Also recall that we should use smoothing when calculating the above probabilities

$P(x_i|y) = \frac{\text{count}(x_i,y) + \alpha}{\sum_{x \in V} \text{count}(x,y)+ \alpha|V|}$ -->


In [ ]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import List, Text

In [ ]:
def create_ngrams(tokens: List[Text], n: int):
  """
    Take a sequence of tokens and return a list of n-grams
  """
  ngrms_list = []
  for i in range(len(tokens) - n + 1):
      ngrms_list.append(tuple(tokens[i:i + n]))
  return ngrms_list


class NGRAM_LM:
  unk = "<UNK>"
  start_token = "<s>"
  end_token = "</s>"

  def __init__(self, n_gram: int, smoothing: bool):

    #n-gram order
    self.n = n_gram
    #enable/disable Laplace smoothing
    self.smoothing = smoothing
    #counter for n-grams (i.e., the numerator to calculate n-gram probs)
    self.n_grams = Counter()
    #counter for n-1-grams (i.e., the denominator to calculate n-gram probs)
    self.n_minus_one_grams = Counter()

  def train(self, training_file_path: Text):
    """
      Train the language model by accumulating counts of n-grams and n-1-grams
      Note: *do not* include the pseudo-counts if smoothing is enabled (we will do this at inference time)
    """
    with open(training_file_path, "r") as f:
      # count the unigrams and keep track of all the words (we may need to replace some of these with unks)
      unigrams = Counter()
      all_tokens = []
      for sentence in f:
        words = sentence.split()
        unigrams.update(words)
        all_tokens += words

      # replace singletons with UNK
      unigrams = Counter([word if unigrams[word] > 1 else self.unk
                          for word in all_tokens])

      self.vocab = list(unigrams.keys())
      self.token_counts = sum(unigrams.values())

      # now collect our ngrams and n-1-gram counts
      # Creating n-grams by replacing unknown words with UNK
      ngrams = create_ngrams([word if unigrams[word] > 1 else self.unk
                              for word in all_tokens], self.n)
      self.n_grams.update(ngrams)

      # Creating n-1-grams by replacing unknown words with UNK
      n_minus_1_grams = create_ngrams([word if unigrams[word] > 1
                                       else self.unk for
                                       word in all_tokens], self.n - 1)
      self.n_minus_one_grams.update(n_minus_1_grams)


    print("Training n-gram:", self.n)
    print("vocab size:", len(self.vocab))
    print("Token Counts:", self.token_counts)
    print("N-gram Counts:", sum(self.n_grams.values()))
    print("Unique n-grams:", len(self.n_grams))
    print("n-1-grams Counts", sum(self.n_minus_one_grams.values()))
    print("Unique n-1-grams:", len(self.n_minus_one_grams))
    print("<UNK> Counts:", unigrams[self.unk])
    print()

  def score(self, sentence: Text):
    """
      Compute the probability of a sequence as a product of individual n-gram probabilities
      (or sum of log probabilities)
      We will use the counts that were accumulated during training to compute the individual n-gram probabilities
      if smoothing is enabled we also need to add the pseudo-counts
      Note that there are two cases: unigrams and n-grams (with n > 1)

    """
    words = sentence.strip().split()
    # making unknown words in vocab as UNK
    words = [self.unk if not word in self.vocab else word for word in words]
    # converting the list to ngrams
    sent_ngrams = create_ngrams(words, self.n)
    total_prob = 0
    # for each n gram calc prob
    for sent_ngram in sent_ngrams:
      # if n=1, then the formula is as follows
      if self.n == 1:
            # Unigram
            count = self.n_grams[sent_ngram]
            # laplace smoothing
            if self.smoothing:
                prob = (count + 1) / (self.token_counts + len(self.vocab))
            else:
                prob = count / self.token_counts
      else:
          # n-gram
          n_minusone_gram = sent_ngram[:-1]
          count = self.n_grams[sent_ngram]
          prev_count = self.n_minus_one_grams[n_minusone_gram]

          if self.smoothing:
              # Laplace smoothing
              prob = (count + 1) / (prev_count + len(self.vocab))
          else:
              prob = count / prev_count if prev_count > 0 else 0

      # taking prob only if its grater than 0 to avoid infinity value for log
      if prob > 0:
          total_prob += np.log(prob)

    return total_prob


  # TODO: implement
  def perplexity(self, sentence: Text):
    """
      Compute the perplexity of a sentence under the model
    """
    return math.exp(-self.score(sentence)/len(sentence.strip().split()))



  def generate(self):
    """
      Generate a sentence using Shannon's method
    """
    num_begin = self.n - 1 if self.n > 1 else 1
    sent = [self.start_token for i in range(num_begin)]
    curr = sent[len(sent) - 1]
    if self.n == 1:
      # remove the <s> from our vocab for unigrams
      lookup = [word for word in self.vocab if word != self.start_token]
      #remove counts of the start of sentence
      token_counts = self.token_counts - self.n_grams[tuple([self.start_token])]
      weights = [(self.n_grams[tuple([word])])/(token_counts) for word in lookup]

    while curr != self.end_token:
      if (self.n > 1):
        # get the n - 1 previous words that we are sampling
        previous = tuple(sent[len(sent) - (self.n - 1) : len(sent)])
        previous_count = self.n_minus_one_grams[previous]

        lookup = [choice for choice in self.n_grams if choice[:-1] == previous]
        weights = [self.n_grams[choice] / previous_count for choice in lookup]

      to_sample = np.arange(len(lookup))
      next = lookup[np.random.choice(to_sample, p=weights)]
      #avoid generating just start and end of sentence
      if next == self.end_token and curr == self.start_token: continue

      if self.n == 1:
        sent.append(next)
      else:
        sent.append(next[-1])
      curr = sent[-1]

    return " ".join(sent)


def score_testfile(lm: NGRAM_LM, test_file_path: Text):
  """
    Compute the probability score for a set of sentences on a test set
    Prints the number of sentences, average probability and standard deviation
  """

  with open(test_file_path, "r", encoding="utf8") as f:
      scores = [lm.score(s.strip()) for s in f.readlines()]

  print("Number of sentences:", len(scores))
  print("Average score:", np.average(scores))
  print("Std deviation:", np.std(scores))
  print()


def train_test_lm(n_gram: int, smooth: bool):
  """
    Train and test an n-gram language mode
  """
  #Paths
  trainingFilePath = "LM-training.txt"
  test_file_path = "LM-test.txt"
  test_sentence = "<s> sam i am and today I am walking away </s>"

  language_model = NGRAM_LM(n_gram, smooth)
  language_model.train(trainingFilePath)

  print("Score on test file")
  score_testfile(language_model, test_file_path)
  print("Probability of test sentence: ", language_model.score(test_sentence))
  print("Perplexity of test sentence ", language_model.perplexity(test_sentence))

  return language_model


Q1: Implement the `create_ngrams()` method which takes as input a string and an `n` parameter and returns a list of `n-grams`

In [ ]:
def create_ngrams(tokens: List[Text], n: int):
  """
    Take a sequence of tokens and return a list of n-grams
  """
  ngrms = []
  for i in range(len(tokens) - n + 1):
    ngrms.append(tuple(tokens[i:i+n]))
  return ngrms

Q2: Implement the `train()` method. We have already implemented code to: count unigram occurrences, replace singleton tokens with UNK, and induce the vocabulary. Now you need read the training data again and count the occurrence of n-grams and n-1-grams. The actual probabilities will be computed at inference time. Do not add the pseudo-counts for smoothing (this will be done when computing the actual probabilities).

In [ ]:
def train(self, training_file_path: Text):
    """
      Train the language model by accumulating counts of n-grams and n-1-grams
      Note: *do not* include the pseudo-counts if smoothing is enabled (we will do this at inference time)
    """
    with open(training_file_path, "r") as f:
      # count the unigrams and keep track of all the words (we may need to replace some of these with unks)
      unigrams = Counter()
      all_tokens = []
      for sentence in f:
        words = sentence.split()
        unigrams.update(words)
        all_tokens += words

      # replace singletons with UNK
      unigrams = Counter([word if unigrams[word] > 1 else self.unk
                          for word in all_tokens])

      self.vocab = list(unigrams.keys())
      self.token_counts = sum(unigrams.values())

      # now collect our ngrams and n-1-gram counts
      # Create n-grams
      ngrams = create_ngrams([word if unigrams[word] > 1 else self.unk
                              for word in all_tokens], self.n)
      self.n_grams.update(ngrams)

      # Create n-1-grams for normalization
      n_minus_1_grams = create_ngrams([word if unigrams[word] > 1
                                       else self.unk
                                       for word in all_tokens], self.n - 1)
      self.n_minus_one_grams.update(n_minus_1_grams)



Q3: Implement the `score()` method which takes a sentence as input and calculates its probability with and without Laplace smoothing. Use the counts from the training step to calculate the individual n-gram probabilities (`self.token_counts` has the sum of the counts of all tokens in the corpus).

Q3.1: Implement the `perplexity()` method (you can use the probabilities from the score method)



In [ ]:
def score(self, sentence: Text):
  """
    Compute the probability of a sequence as a product of individual n-gram probabilities
    (or sum of log probabilities)
    We will use the counts that were accumulated during training to compute the individual n-gram probabilities
    if smoothing is enabled we also need to add the pseudo-counts
    Note that there are two cases: unigrams and n-grams (with n > 1)

  """
  words = sentence.strip().split()
  # making unknown words in vocab as UNK
  words = [self.unk if not word in self.vocab else word for word in words]
  # converting the list to ngrams
  sent_ngrams = create_ngrams(words, self.n)
  total_prob = 0
  # for each n gram calc prob
  for sent_ngram in sent_ngrams:
    # if n=1, then the formula is as follows
    if self.n == 1:
          # Unigram
          count = self.n_grams[sent_ngram]
          # laplace smoothing
          if self.smoothing:
              prob = (count + 1) / (self.token_counts + len(self.vocab))
          else:
              prob = count / self.token_counts
    else:
        # n-gram
        n_minusone_gram = sent_ngram[:-1]
        count = self.n_grams[sent_ngram]
        prev_count = self.n_minus_one_grams[n_minusone_gram]

        if self.smoothing:
            # Laplace smoothing
            prob = (count + 1) / (prev_count + len(self.vocab))
        else:
            prob = count / prev_count if prev_count > 0 else 0

    # taking prob only if its grater than 0 to avoid infinity value for log
    if prob > 0:
        total_prob += np.log(prob)

  return total_prob


def perplexity(self, sentence: Text):
  """
    Compute the perplexity of a sentence under the model
  """
  len_of_sentence = len(sentence.strip().split())+2 # start and end token
  return math.exp(-self.score(sentence)/len(sentence.strip().split()))

Q4: Use the `train_test_lm()` method to evaluate a unigram and a bigram LM (with and without smoothing). What do you observe?

In [ ]:
print('\nunigram without smoothing')
unigram_lm = train_test_lm(1, False)
print('\nbigram without smoothing')
bigram_lm = train_test_lm(2, False)

print('\nunigram with smoothing')
unigram_lm_smooth = train_test_lm(1, True)
print('\nbigram with smoothing')
bigram_lm_smooth = train_test_lm(2, True)


unigram without smoothing
Training n-gram: 1
vocab size: 923
Token Counts: 56696
N-gram Counts: 56696
Unique n-grams: 923
n-1-grams Counts 56697
Unique n-1-grams: 1
<UNK> Counts: 478

Score on test file
Number of sentences: 101
Average score: -37.21092847483953
Std deviation: 19.3752306064226

Probability of test sentence:  -57.634641729807186
Perplexity of test sentence  188.57822065999315

bigram without smoothing
Training n-gram: 2
vocab size: 923
Token Counts: 56696
N-gram Counts: 56695
Unique n-grams: 8289
n-1-grams Counts 56696
Unique n-1-grams: 923
<UNK> Counts: 478

Score on test file
Number of sentences: 101
Average score: -15.347592920219412
Std deviation: 7.58723791411062

Probability of test sentence:  -15.150356455236562
Perplexity of test sentence  3.9642042139962927

unigram with smoothing
Training n-gram: 1
vocab size: 923
Token Counts: 56696
N-gram Counts: 56696
Unique n-grams: 923
n-1-grams Counts 56697
Unique n-1-grams: 1
<UNK> Counts: 478

Score on test file
Number

Q5: Use the `generate()` method to generate samples from your language models. Compare the samples from a unigram and bigram language models. What do you notice?



In [ ]:
print("Generate sentences:")
N_SENTS=5

print("Bigrams without smoothing")
bigram_lm_generations = [bigram_lm.generate() for i in range(N_SENTS)]
print(bigram_lm_generations)

print("Bigrams with smoothing")
bigram_lm_generations_smoothed = [bigram_lm_smooth.generate()
                                          for i in range(N_SENTS)]
print(bigram_lm_generations_smoothed)

print("unigrams without smoothing")
unigram_lm_generations = [unigram_lm.generate() for i in range(N_SENTS)]
print(unigram_lm_generations)

print("unigrams with smoothing")
unigram_lm_generations_smoothed = [unigram_lm_smooth.generate()
                                                    for i in range(N_SENTS)]
print(unigram_lm_generations_smoothed)



Generate sentences:
Bigrams without smoothing
['<s> tell me a distance </s>', '<s> is your database </s>', "<s> what's the list again </s>", '<s> i can you have lunch </s>', '<s> where can you to eat dinner </s>']
Bigrams with smoothing
['<s> kosher </s>', "<s> could you show me about greek restaurants around icsi can you know about lunch should be at bucci's menu of southern style restaurant </s>", '<s> ten dollars </s>', '<s> start over please </s>', "<s> show me that right can be at any type of let's try again </s>"]
unigrams without smoothing
['<s> want the on restaurant reasonably want i lunch should help like show um as i some within </s>', '<s> is </s>', "<s> twenty oh actually house okay any the is doesn't i food try takes you you </s>", '<s> out desserts start </s>', '<s> how house what be something </s>']
unigrams with smoothing
['<s> restaurant list the japanese it very </s>', "<s> mcdonald's like okay return lunch i over give about i </s>", '<s> are californian again </s>',

I have observed the following things:

### Bigram Model
- **Without Smoothing**: The sentences are understandable but some are not proper and not sound natural.

- **With Smoothing**: The sentences sound more natural and the flow is good.

### Unigram Model
- **Without Smoothing**: The sentences look like random words

- **With Smoothing**: The sentences are still random, but smoothing adds more variety than before

### To Conclude
- **Bigram models** did a better job of creating sentences because they consider the word before the current one.
- **Unigram models** produced random sentences because they don’t think about the order of words and do not have any context.
- **Smoothing** helped both models by making the sentences less repetitive and allowing them to handle words they haven’t seen together before.

In short, the bigram model with smoothing gives the best sentences than unigram.